In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## SQLChatMessageHistory란?
LangChain에서 제공하는 대화 히스토리 저장소 클래스 중 하나.  
대화 기록을 SQLite 같은 RDBMS에 저장하는 역할  
- 사용자별 대화 히스토리를 DB에 영구 저장
- 히스토리를 LLM 체인에 자동으로 불러오거나 갱신할 수 있게 한다.

In [3]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="sql_history_test",
    connection="sqlite:///sqlite_basic.db"
)

In [4]:
from langchain_core.messages import HumanMessage

chat_message_history.add_message(HumanMessage("hello"))
chat_message_history.messages

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

In [5]:
chat_message_history.add_user_message(
    "안녕 지피티야 나는 은빈이야. 오늘 날씨가 어떄?"
)

In [6]:
chat_message_history.messages

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕 지피티야 나는 은빈이야. 오늘 날씨가 어떄?', additional_kwargs={}, response_metadata={})]

In [7]:
chat_message_history.add_ai_message(
    "비가 와"
)
chat_message_history.messages

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕 지피티야 나는 은빈이야. 오늘 날씨가 어떄?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='비가 와', additional_kwargs={}, response_metadata={})]

# LLM이랑 연동

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

In [11]:
# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요."),
        # 여기서부터 대화 내용 저장
        MessagesPlaceholder(variable_name="chat_history"), # <- 여기에 지금까지의 대화내용(chat_history)이 자동으로 삽입됨. 
                                                        # variable_name: LangChain에서 RunnableWithMessageHistory 쓸 때 자동으로 매핑되는 변수
        ("human", "#Question:\n{question}")
    ]
)

- LLM이나 chain과 연결하면 이전 대화 + 현재 질문을 바탕으로 문맥있는 응답을 생성할 수 있게 됨.
- 따라서 프롬프트 토큰 수가 계속 증가함

In [12]:
chain = prompt | model | StrOutputParser()

- 유저별, 대화별로 내용 저장
- user_id: 사용자 ID
- conversation_id: 대화 ID

In [13]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",                        # id 필드의 내부 식별자
        annotation=str,                 # 기대하는 데이터 타입
        name="User ID",                # 나중에 보기 위한 기록을 위한 값
        description="유저별로 식별",       # 이 항목(필드)에 대한 설명
        default="",                     # 기본값은 빈 문자열
        is_shared=True,                 # 다른 곳에서도 실행할 때 공유가능
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="대화별 식별",
        default="",
        is_shared=True
    )
]

In [14]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# 대화 내용을 sqlite에서 조회해서 가져오기, sqlite에 저장
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite_basic2.db"
    )

In [16]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="question",
    history_messages_key="chat_history",
    history_factory_config=config_fields
)

In [17]:
config = {
    "configurable": {
        "user_id": "binmuxiz",
        "conversation_id": "conversation1"
    }
}

result = chain_with_history.invoke({
    'question': "안녕 나는 은빈이야. 만나서 반가워"
}, config)

In [18]:
result

'안녕하세요, 은빈님! 만나서 반가워요. 오늘 어떻게 도와드릴까요?'

In [19]:
config = {
    "configurable": {
        "user_id": "binmuxiz",
        "conversation_id": "conversation1"
    }
}

result = chain_with_history.invoke({
    'question': "내 이름이 뭐게"
}, config)

result

'당신의 이름은 은빈이라고 하셨죠! 맞나요?'

In [20]:
config = {
    "configurable": {
        "user_id": "sense",
        "conversation_id": "conversation1"
    }
}

result = chain_with_history.invoke({
    'question': "안녕 내 이름은 센세야"
}, config)

result

'안녕하세요, 센세! 만나서 반가워요. 어떻게 도와드릴까요?'

In [21]:
config = {
    "configurable": {
        "user_id": "sense",
        "conversation_id": "conversation2"
    }
}

result = chain_with_history.invoke({
    'question': "안녕 내 이름은 센세야"
}, config)

result

'안녕하세요, 센세! 만나서 반가워요. 어떻게 도와드릴까요?'

In [22]:
user1_conv1_history = get_chat_history(user_id="binmuxiz", conversation_id="conversation1")
print(type(user1_conv1_history))
for msg in user1_conv1_history.messages:
    print(msg)

<class 'langchain_community.chat_message_histories.sql.SQLChatMessageHistory'>
content='안녕 나는 은빈이야. 만나서 반가워' additional_kwargs={} response_metadata={}
content='안녕하세요, 은빈님! 만나서 반가워요. 오늘 어떻게 도와드릴까요?' additional_kwargs={} response_metadata={}
content='내 이름이 뭐게' additional_kwargs={} response_metadata={}
content='당신의 이름은 은빈이라고 하셨죠! 맞나요?' additional_kwargs={} response_metadata={}
